In [2]:
import requests
from bs4 import BeautifulSoup
import unicodedata
import pandas as pd

# Static URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)

# Create BeautifulSoup object
soup = BeautifulSoup(response.text, 'html.parser')

# ------------------- Helper Functions ---------------------

def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    return ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i % 2 == 0][0:-1])

def landing_status(table_cells):
    return [i for i in table_cells.strings][0]

def get_mass(table_cells):
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if "kg" in mass:
        return mass[:mass.find("kg") + 2]
    return "0"

def extract_column_from_header(row):
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    colunm_name = ' '.join(row.contents)
    if not(colunm_name.strip().isdigit()):
        return colunm_name.strip()

# ------------------- Setup Launch Dictionary ---------------------

column_names = [
    'Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer',
    'Launch outcome', 'Version Booster', 'Booster landing', 'Date', 'Time'
]

launch_dict = {key: [] for key in column_names}

# ------------------- Parse Table ---------------------

for table in soup.find_all('table', "wikitable plainrowheaders collapsible"):
    for rows in table.find_all("tr"):
        if rows.th and rows.th.string and rows.th.string.strip().isdigit():
            flight_number = rows.th.string.strip()
            row = rows.find_all('td')
            if len(row) >= 9:  # ensure there are enough cells
                # Parse each value
                datatimelist = date_time(row[0])
                date = datatimelist[0].strip(',')
                time = datatimelist[1]

                bv = booster_version(row[1])
                if not bv and row[1].a:
                    bv = row[1].a.string

                launch_site = row[2].a.string if row[2].a else row[2].text.strip()
                payload = row[3].a.string if row[3].a else row[3].text.strip()
                payload_mass = get_mass(row[4])
                orbit = row[5].a.string if row[5].a else row[5].text.strip()
                customer = row[6].a.string if row[6].a else row[6].text.strip()
                launch_outcome = list(row[7].strings)[0]
                booster_landing = landing_status(row[8])

                # Append to dictionary
                launch_dict['Flight No.'].append(flight_number)
                launch_dict['Date'].append(date)
                launch_dict['Time'].append(time)
                launch_dict['Version Booster'].append(bv)
                launch_dict['Launch site'].append(launch_site)
                launch_dict['Payload'].append(payload)
                launch_dict['Payload mass'].append(payload_mass)
                launch_dict['Orbit'].append(orbit)
                launch_dict['Customer'].append(customer)
                launch_dict['Launch outcome'].append(launch_outcome)
                launch_dict['Booster landing'].append(booster_landing)

# ------------------- Convert to DataFrame ---------------------

df = pd.DataFrame(launch_dict)
df.to_csv('spacex_web_scraped.csv', index=False)
print("Saved to spacex_web_scraped.csv")


Saved to spacex_web_scraped.csv


In [3]:
import requests
from bs4 import BeautifulSoup

static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)
soup = BeautifulSoup(response.text, 'html.parser')

print(soup.title)


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>
